# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Csv file 
weather_df_csv = '../Output/weather_df.csv'
weather_df = pd.read_csv(weather_df_csv)
dropna = weather_df.dropna()
del dropna["Unnamed: 0"]
dropna.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.53,20.04,288.15,58,100,4.60,ZA,1603143847
1,korla,41.76,86.15,280.37,79,0,0.45,CN,1603143847
2,atuona,-9.80,-139.03,298.96,76,0,7.78,PF,1603143847
3,brigham city,41.51,-112.02,291.40,25,1,4.67,US,1603143848
4,arraial do cabo,-22.97,-42.03,296.15,78,40,9.80,BR,1603143848


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# locations
loc = dropna[['Lat', 'Lng']]
humidity = dropna['Humidity'].astype(float)

In [13]:
# Plot Heatmap
Heatmap = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(loc, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=4)

# Heatmap layer
Heatmap.add_layer(heat_layer)

Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
